<a href="https://colab.research.google.com/github/jadevaibhav/UNet-for-biomedical-images/blob/master/Unet_brats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os 
os.chdir('drive/My Drive/brat')

In [0]:
import keras
import tensorflow as tf
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Lambda,Concatenate,Cropping2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, Add, Conv2DTranspose
from keras.models import Model
from keras import regularizers
from keras.preprocessing import image
from keras.utils import layer_utils,to_categorical
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.initializers import glorot_normal
#import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

**No need of cropping with same padding**

In [0]:
def crop(in1,in2):
  len1 = int(in1[1])*2
  wid1 = int(in1[2])*2
  len2 = int(in2[1])
  wid2 = int(in2[2])
  ln = int((len2-len1)/2) 
  wid = int((wid2-wid1)/2)
  return (ln,wid)

In [0]:
def conv_block(X,filters):
  X = Conv2D(filters,(3,3),padding='same')(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  X = Conv2D(filters,(3,3),padding = 'same')(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  
  return X

In [0]:
def down_block(X,block_no):
  Xa = conv_block(X,64*2**(block_no - 1))
  X = MaxPooling2D((2,2))(Xa)
  print(X.shape)
  
  return(X,Xa)

In [0]:
def up_block(X,Xa):
  
  channels = int(Xa.shape[3])
  X = Conv2DTranspose(channels,(2,2),strides=(2,2))(X)
  X = Concatenate(axis=-1)([Xa,X])
  X = conv_block(X,channels)
  print(X.shape)
  
  return X

In [0]:
  input_shape=(None,None,4)
#def unet(input_shape):
  X_in = Input(input_shape)
  
  X,X1 = down_block(X_in,1)
  
  X,X2 = down_block(X,2)
    
  X,X3 = down_block(X,3)
  
  _,X = down_block(X,4)
   
  X = up_block(X,X3)
  
  X = up_block(X,X2)
  
  X = up_block(X,X1)
  
  X = Conv2D(5,(1,1))(X)
  #X = Activation('softmax')(X)
  print(X.shape)

  M = Model(inputs=X_in,outputs=X)

(?, ?, ?, 64)
(?, ?, ?, 128)
(?, ?, ?, 256)
(?, ?, ?, 512)
(?, ?, ?, 256)
(?, ?, ?, 128)
(?, ?, ?, 64)
(?, ?, ?, 5)


In [0]:
M.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, None, None, 4 0                                            
__________________________________________________________________________________________________
conv2d_46 (Conv2D)              (None, None, None, 6 2368        input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_43 (BatchNo (None, None, None, 6 256         conv2d_46[0][0]                  
__________________________________________________________________________________________________
activation_45 (Activation)      (None, None, None, 6 0           batch_normalization_43[0][0]     
__________________________________________________________________________________________________
conv2d_47 

In [0]:
def sparse_loss(y_true, y_pred):
    return tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true,
                                                          logits=y_pred)

In [0]:
M.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

**Data processing**

In [0]:
!pip3 install SimpleITK

    100% |████████████████████████████████| 41.0MB 764kB/s 


In [0]:
from sklearn.utils import class_weight

In [0]:
import SimpleITK as sitk
import numpy as np

In [0]:
fold = os.listdir('HG/')
fold.sort(key=str.lower) 

for path in fold:
    print(path)
    path = 'HG/'+path
    p = os.listdir(path)
    p.sort(key=str.lower)
    arr = []
    
    # Reading from 4 images and creating 4 channel slice-wise 
    for i in range(len(p)):
      if(i != 4):
        p1 = os.listdir(path+'/'+p[i])
        p1.sort()
        img = sitk.ReadImage(path+'/'+p[i]+'/'+p1[-1])
        arr.append(sitk.GetArrayFromImage(img))
      else:
        p1 = os.listdir(path+'/'+p[i])
        img = sitk.ReadImage(path+'/'+p[i]+'/'+p1[0])
        Y_labels = sitk.GetArrayFromImage(img)
    data = np.zeros((Y_labels.shape[0],Y_labels.shape[1],Y_labels.shape[2],4))
    for i in range(Y_labels.shape[0]):
      data[i,:,:,0] = arr[0][i,:,:]
      data[i,:,:,1] = arr[1][i,:,:]
      data[i,:,:,2] = arr[2][i,:,:]
      data[i,:,:,3] = arr[3][i,:,:]
    Y_labels = Y_labels.reshape((Y_labels.shape[0],Y_labels.shape[1],Y_labels.shape[2],1))
    y = Y_labels.reshape((-1))
    class_weights = class_weight.compute_class_weight('balanced',
                                                          np.unique(y),
                                                          y)
    Y_labels = to_categorical(Y_labels,5)
    print(data.shape)
    print(Y_labels.shape)
    # Training the model
    M.fit(data,Y_labels,epochs=5,batch_size=4,class_weight = class_weights)
    M.save('unet_brats.h5')

0001
(176, 216, 160, 4)
(176, 216, 160, 5)
Epoch 1/5
176/176 [==============================] - 742s 4s/step - loss: nan - acc: 0.9816
Epoch 2/5
176/176 [==============================] - 742s 4s/step - loss: nan - acc: 0.9816
Epoch 3/5
172/176 [============================>.] - ETA: 16s - loss: nan - acc: 0.9814